In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2023-08-05


Found rundate 2023-08-04 10:00:03 < 2023-08-05 at commit 5ec29908; returning commit 85fb120


'2023-08-07 10:00:03'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2023-08-07 10:00:03, NJSP was reporting 324 YTD deaths
Current YTD Deaths (2024-08-05 10:00:01-04:00): 359
Previous year YTD Deaths (adjusted; 2023-08-07 10:00:03): 321.63508755123183
Projected 2024 total: 664.0541842373398
11.6% change


FAUQStats cache miss: 2023, Mon Aug 07 10:00:03 EDT 2023


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,STREET,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,HIGHWAY,INJURIES,dt
ACCID,,,,,,,,,,,,,,
12249,07,Essex,0714,Newark City,Mulberry St,Mulberry St,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2023-01-01 01:49:00-05:00
12251,01,Atlantic,0112,Hamilton Twsp,NaN,State Highway 322 MP 46.4,1.0,1.0,0.0,0.0,0.0,322,0.0,2023-01-02 09:26:00-05:00
12253,12,Middlesex,1217,Piscataway Twsp,South Randolphville Rd,South Randolphville Rd,1.0,1.0,0.0,0.0,0.0,NaN,0.0,2023-01-02 19:41:00-05:00
12254,02,Bergen,0221,Garfield City,Outwater Ln,Outwater Ln,1.0,0.0,0.0,1.0,0.0,NaN,0.0,2023-01-03 20:52:00-05:00
12267,07,Essex,0714,Newark City,Washington St,Washington St,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2023-01-04 06:21:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12592,16,Passaic,1608,Paterson City,NaN,Interstate 80 MP 58.7,1.0,1.0,0.0,0.0,0.0,80,NaN,2023-07-31 21:48:00-04:00
12595,15,Ocean,1511,Jackson Twsp,Don Connor Blvd,Don Connor Blvd,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2023-08-02 10:38:00-04:00
12599,15,Ocean,1518,Manchester Twsp,NaN,State Highway 37,1.0,1.0,0.0,0.0,0.0,37,NaN,2023-08-03 22:04:00-04:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,18,1,11,2,31
Bergen,11,3,10,1,23
Burlington,19,4,2,1,26
Camden,12,3,11,0,26
Cape May,2,0,0,0,2
Cumberland,8,2,2,0,11
Essex,10,3,16,0,29
Gloucester,7,2,0,0,9
Hudson,4,2,1,0,6


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

59.4% through the year, 40.6% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,10,18,18
Bergen,driver,14,20,11
Burlington,driver,12,26,19
Camden,driver,9,18,12
Cape May,driver,3,4,2
...,...,...,...,...
Salem,crashes,6,11,8
Somerset,crashes,8,21,9
Sussex,crashes,1,6,8


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,52,4,30,3,21
Bergen,37,2,16,6,18
Burlington,53,2,38,7,4
Camden,59,2,23,6,32
Cape May,4,0,3,0,0
Cumberland,19,0,15,3,3
Essex,54,0,16,5,34
Gloucester,15,0,12,3,1
Hudson,14,1,8,4,3


In [9]:
projected.to_csv(PROJECTED_CSV)